This script maps the PC Ibex result files (one for each list) onto a table with subject information.
Mostly for approving / rejecting HITs

- subject_id used for filenames (4 alphabets)
- the name of zip files
- Unique Identifiers (8 alphabets/numbers)
- Worker ID/Name

*The names of the result csv files must end with the list id*

In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
# Get the result directory
# The last character/number must indicate the name of the latin-square list
result_dir = "/Users/masato/Box/cloze_experiments/filter/batch2/results"

# Set the output path
output_path = "/Users/masato/Box/cloze_experiments/filter/batch2/results/subj_table.csv"

In [3]:
# Get the path of the result files
file_list = [f for f in os.listdir(result_dir) if "results" in f]
path_list = [os.path.join(result_dir, f) for f in file_list]

In [4]:
# Get the relevant raws for all the lists / experiments

idt_list_all = [] # identifier
wid_list_all = [] # worker id
fn_list_all = [] # zipfile names
ls_list = [] # list id
subjid_list_all = [] #subject id

for p in path_list:
    with open(p) as f:
        raw = f.readlines()
    list_id = os.path.splitext(os.path.basename(p))[0][-1]
    rel1 = [i for i in raw if "exit_form" in i or "DYNAMIC" in i]
    # Get the relevant raws
    idt_list_raw = [r.split(",") for r in [i for i in rel1 if "identifier" in i]]
    idt_list_all = idt_list_all + idt_list_raw
    
    wid_list_raw = [r.split(",") for r in [i for i in rel1 if "worker_id" in i]]
    wid_list_all = wid_list_all + wid_list_raw
    
    fn_list_raw = [r.split(",") for r in [i for i in rel1 if "Filename" in i]]
    fn_list_all = fn_list_all + fn_list_raw
    
    ls_list = ls_list + [list_id] * len(fn_list_raw)
        
    subjid_list_raw = [r.split(",") for r in [l for l in raw if "exp_60" in l]]
    subjid_list_all = subjid_list_all + subjid_list_raw


In [5]:
# Make sure the time and the ip address is lined up
time_list = [idt_list_all[i][0] for i in range(0, len(idt_list_all))]
time_list1 = [wid_list_all[i][0] for i in range(0, len(wid_list_all))]
time_list2 = [fn_list_all[i][0] for i in range(0, len(fn_list_all))]

ip_list = [idt_list_all[i][1] for i in range(0, len(idt_list_all))]
ip_list1 = [wid_list_all[i][1] for i in range(0, len(wid_list_all))]
ip_list2 = [fn_list_all[i][1] for i in range(0, len(fn_list_all))]

print(time_list == time_list1 and time_list == time_list2)
print(ip_list == ip_list1 and ip_list == ip_list2)

True
True


In [6]:
# Convert times as strings to times as integers
time_list_i = [int(i) for i in time_list]

In [7]:
# Get all the info
idt_list = [idt_list_all[i][9] for i in range(0, len(idt_list_all))]
wid_list = [wid_list_all[i][9] for i in range(0, len(wid_list_all))]
fn_list = [fn_list_all[i][9] for i in range(0, len(fn_list_all))]

subjid_list = [subjid_list_all[i][11] for i in range(0, len(subjid_list_all))]
subjid_list_c = [subjid_list_all[i][7].split("_")[-1] for i in range(0, len(subjid_list_all))]

subjid_list == subjid_list_c

True

In [8]:
# Create a table for all the info
subj_table = pd.DataFrame({"time": time_list_i, "ip": ip_list, "identifier":idt_list, "worker_id": wid_list, "subject_id":subjid_list, "filename": fn_list, "list": ls_list})

In [9]:
subj_table

,time,ip,identifier,worker_id,subject_id,filename,list
0,1634674061,a478f9eac069713bf8bec8cc709b8cc9,123hjk!@#JKL,A5WWHKD82I8UE,ratq,9c098a92-e7ae-b684-47f1-876d10cc64d8.zip,a
1,1634773474,ea56f97381bd5ccffc3bbb2bcf6e2d67,T1dN4K8q,A146MSML39KY98,qulh,b2b2af9f-d52b-5148-16f5-471b751226f1.zip,a
2,1634919259,3695c6300e331dd45516f682886426eb,9zRROAlz,A1IFIK8J49WBER,qwmi,e618810b-6ba2-e92c-3fd1-443fecec81a1.zip,a
3,1634919674,d951c4539865f964caf92aaa45953e42,ZITu6LRU,A68J20100VW4W,zpfd,02fcdf1c-32db-12b8-af82-7a1db3736b29.zip,a
4,1634921014,349837944bd804d95ca74fb8f0299813,EBqRuITN,A002160837SWJFPIAI7L7,akjb,dbab31dd-be06-5afa-e075-fc1ebccb8d6a.zip,a
...,...,...,...,...,...,...,...
80,1636515361,702a87e3337142f34cc0a183ce4b1775,8144RD3,AWKP0M7G5BBCC,cnis,a8b16360-444c-fbd8-fa74-df71e24b40c4.zip,b
81,1636516009,0c6e919577d696c568efdfb9fff8118b,M6U31UPK,A2YW35WSBU88O0,ldan,bd0e9554-e009-0a5c-015e-e5bdbfcbff98.zip,b
82,1636516188,2a72860edc814f6d673d8e9ab761c9f9,Fr6yJot4,A11DLGQTOOSIWR,rmht,01fe8de8-9870-c8ef-e44b-69863cca28b2.zip,b
83,1636516217,e36229b29e6c03a51ed752bcb5f71fd5,FTe1isBP,A17TKHT8FEVH0R,ydyq,a3cf5e3e-61b1-1bec-657f-398085fab0bd.zip,b


In [10]:
# See if there are multiple submissions from the same IP address
if len(set(subj_table.ip)) != len(subj_table):
    import collections
    print(collections.Counter(subj_table.ip).most_common())

else:
    print("No multiple submissions")


No multiple submissions


In [ ]:
# Output the result into a csv file
#subj_table.to_csv(output_path, index = False)

### Add batch info based on the reception time

In [11]:
boundaries = [1634850000, 1636000000]

In [12]:
subj_table["batch"] = "batch2"

In [13]:
subj_table.loc[subj_table["time"] < boundaries[0], ["batch"]] = "pilot"
subj_table.loc[(subj_table["time"] > boundaries[0]) & (subj_table["time"] < boundaries[1]), ["batch"]] = "batch1"

In [14]:
# Output the result into a csv file
subj_table.to_csv(output_path, index = False)

### Use MTurk result files to add batch information and check submissions

In [17]:
amt_res_dir = "/Users/masato/Box/cloze_experiments/AMT/result_files"
id_string = "q_masato_"

In [18]:
amt_res_p = [os.path.join(amt_res_dir, fn) for fn in os.listdir(amt_res_dir) if id_string in fn]

In [20]:
rcolnames = ['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Input.url',
       'Answer.Code', 'Answer.FileName', 'Answer.Identifier', 'Answer.Name',
       'Approve', 'Reject', 'UPDATE-Already participated (WYCL)'] + ["BatchId"]

In [21]:
amt_result = None
for p in amt_res_p:
    t_amt_result = pd.read_csv(p)
    batchid = os.path.splitext(p)[0].split("_")[-1]
    t_amt_result["BatchId"] = batchid
    
    if "amt_result" in locals():
        amt_result = pd.concat([amt_result, t_amt_result])
    else:
        amt_result = t_amt_result


In [22]:
amtset = set(amt_result["WorkerId"])
pcibexset = set(subj_table["worker_id"])

In [23]:
not_in_pcibex = amtset - pcibexset
not_in_amt = pcibexset - amtset

In [24]:
not_in_pcibex

{'A2541C8MY0BYV3', 'AC5UD8N187QD6'}

In [25]:
not_in_amt

{'A2CY1TSQ3KKP5Q'}

In [26]:
subj_table[subj_table["worker_id"].isin(not_in_amt)]

,time,ip,identifier,worker_id,subject_id,filename,list,batch
68,1636489502,7eb9a96eba14627d4513f5c083a181ef,VF1227,A2CY1TSQ3KKP5Q,glxa,3cdbc307-7818-12f6-7ad9-f471d527768b.zip,b,batch2


In [27]:
amt_result[amt_result["WorkerId"].isin(not_in_pcibex)]

,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,...,Last7DaysApprovalRate,Input.url,Answer.Code,Answer.FileName,Answer.Identifier,Answer.Name,Approve,Reject,UPDATE-Already participated (WYCL),BatchId
0,3CRWSLD91J3Z00ZD8ON2CYN2KZWOMF,3BS7Z4CKG2IL4K9Q0W8XGEYY8OBQYP,Try to guess the next word! (audio recording s...,30-minute task: Read incomplete sentences and ...,"Linguistics, Science, Research, University of ...",$6.00,Tue Oct 19 12:23:21 PDT 2021,1,BatchId:4580488;OriginalHitTemplateId:920937340;,3600,...,100% (1/1),https://farm.pcibex.net/p/sguzME/,no code received,{},powIEz0Z,nj,NaN,NaN,1,pilot
5,3TTPFEFXCSJNJGSYHVLV3TTFID3H6A,3BS7Z4CKG2IL4K9Q0W8XGEYY8OBQYP,Try to guess the next word! (audio recording s...,30-minute task: Read incomplete sentences and ...,"Linguistics, Science, Research, University of ...",$6.00,Tue Oct 19 12:23:21 PDT 2021,2,BatchId:4580488;OriginalHitTemplateId:920937340;,3600,...,0% (0/1),https://farm.pcibex.net/p/sguzME/,A2541C8MY0BYV3,{},A2541C8MY0BYV3,BC,NaN,NaN,1,pilot


In [34]:
b2 = subj_table[subj_table["batch"] == "batch2"]

In [72]:
checklist = []
for w in subj_table.worker_id:
    
    if np.sum(amt_result["WorkerId"] == w) == 0:
        checklist.append(w)
    else:
        amt_id = amt_result[amt_result["WorkerId"] == w]["Answer.Identifier"].iloc[0]
        pc_id = subj_table[subj_table["worker_id"] == w]["identifier"].iloc[0]
    
        if amt_id != pc_id:
            checklist.append(w)

In [73]:
checklist

['A68J20100VW4W', 'A2CY1TSQ3KKP5Q']

In [75]:
subj_table[subj_table["worker_id"].isin(checklist)]

,time,ip,identifier,worker_id,subject_id,filename,list,batch
3,1634919674,d951c4539865f964caf92aaa45953e42,ZITu6LRU,A68J20100VW4W,zpfd,02fcdf1c-32db-12b8-af82-7a1db3736b29.zip,a,batch1
68,1636489502,7eb9a96eba14627d4513f5c083a181ef,VF1227,A2CY1TSQ3KKP5Q,glxa,3cdbc307-7818-12f6-7ad9-f471d527768b.zip,b,batch2


In [78]:
amt_result[amt_result["WorkerId"].isin(checklist)]

,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,...,Last7DaysApprovalRate,Input.url,Answer.Code,Answer.FileName,Answer.Identifier,Answer.Name,Approve,Reject,UPDATE-Already participated (WYCL),BatchId
12,35U0MRQMUI6IQNTMMGOTZU6FBV1VO2,3X2H010HBRVVF0VJKQQ2C07YPVIR2A,Try to guess the next word! (audio recording s...,30-minute task: Read incomplete sentences and ...,"Linguistics, Science, Research, University of ...",$6.00,Fri Oct 22 08:38:30 PDT 2021,5,BatchId:4583144;OriginalHitTemplateId:920937340;,5400,...,100% (1/1),https://farm.pcibex.net/p/sguzME/,ZlTu6LRU,{},nD3i9E,ml,NaN,NaN,1,batch1
